In [11]:
%matplotlib inline

import librosa
import soundfile as sf
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt  
import os
import pickle

STRIDE_SIZE = 10.
FRAME_SIZE = 25.
N_MFCC = 16

WINDOW = 1000
WINDOW_STRIDE = 30

def find_peaks(X, Y, iterations=1):
    out = list(zip(X, Y))
    for i in range(iterations):
        temp_out = []
        for j in range(len(out)):
            if (j-1) >= 0 and (j+1) < len(out):
                if out[j-1][1] < out[j][1] and out[j][1] < out[j+1][1]:
                    temp_out.append(out[j])
        out = temp_out
    return out

def truncate_peaks(peaks, thresh = 5):
    curr = peaks[0]
    keep = []
    for p in peaks:
        if abs(p[0] - curr[0]) >= thresh:
            keep.append(curr)
        curr = p
    return keep
    
def window_number_to_sec(win_num, win_len = WINDOW):
    stamp = win_num + (win_len // 2)
    return (FRAME_SIZE + STRIDE_SIZE * (stamp - 1)) / 1000.

def get_mfcc_features(song_data, sample_rate, stride_size = STRIDE_SIZE, frame_size = FRAME_SIZE):
    mfccs = librosa.feature.mfcc(song_data, sample_rate, 
                                 n_mfcc=N_MFCC,
                                 hop_length=int(STRIDE_SIZE / 1000. * sample_rate), 
                                 n_fft=int(FRAME_SIZE / 1000. * sample_rate))
    return mfccs

def load_song(song_file):
    data, samplerate = sf.read(song_file)
    data = data[:, 0]
    mfccs = get_mfcc_features(data, samplerate)
    mfccs = np.asarray(mfccs).T
    return mfccs

def get_bpm_resp_window(song_data, sr):
    onset_env = librosa.onset.onset_strength(song_data, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    bps = tempo / 60.0
    return 32.0 / bps
    

def continuous_prob_estimate(mfccs):
    low_score = 1e9
    low_score_index = -1

    X = []
    Y = []
    print(np.shape(mfccs))
    for i in range(0, np.shape(mfccs)[0], WINDOW_STRIDE):
        if np.shape(mfccs)[0] - WINDOW <= i:
            break
        model = hmm.GaussianHMM(n_components=1	)
        model = model.fit(mfccs[i:i+WINDOW//2], [WINDOW//2])
        curr_score = model.score(mfccs[i+WINDOW//2:i+WINDOW], [len(mfccs[i+WINDOW//2:i+WINDOW])])
        X.append(window_number_to_sec(i))
        Y.append(curr_score * -1)
        if curr_score < low_score:
            low_score = curr_score
            low_score_index = i
    return X, Y
            
def make_peak_plot(X, Y, n=3):
    peaks = find_peaks(X, Y, iterations=n)
    trunc_peaks = truncate_peaks(peaks)
    #trunc_peaks = peaks
    print(trunc_peaks)
    plt.plot(X, Y)
    # for p in peaks:
    #     plt.axvline(x=p[0], color="red")
    for p in trunc_peaks:
        plt.axvline(x=p[0], color="red")

    plt.show()

In [9]:
from pathlib import Path
home = str(Path.home())
SONG_DIR = home + "/Downloads/songdata_90/songdata/"

results = []

for filename in os.listdir(SONG_DIR):
    if ".wav" in filename:
        mfccs = load_song(SONG_DIR + filename)
        X, Y = continuous_prob_estimate(mfccs)
        peaks = find_peaks(X, Y, iterations=3)
        trunc_peaks = truncate_peaks(peaks)
        print(filename,"\n", trunc_peaks)
        results.append((filename, trunc_peaks))

result_file = open('resultsFile', 'wb')   
# source, destination 
pickle.dump(results, result_file)                      
result_file.close() 

Galantis___Love_on_Me.wav 
 [(17.315, 34659.30334961195), (31.715, 32410.139333124494), (48.515, 34085.47863478417), (62.915, 37635.701748140666), (78.515, 33658.37631725218), (99.515, 41170.46241034264), (119.315, 35474.962074381736), (132.815, 31904.734328831742), (151.715, 39224.98347038537), (167.315, 32690.811354184912), (178.415, 31541.050300011666), (187.415, 34979.46682259532)]
Michael_Jackson___Love_Never_Felt_So_Good_(Fedde_Le_Grand_Remix).wav 
 [(9.515, 31307.386476111573), (29.615, 29876.261635858365), (44.315, 33004.175525182145), (52.415, 31512.828665511443), (72.215, 34585.41579742283), (92.315, 28001.037797311696), (100.415, 29376.16886319452), (119.615, 47088.9686350784), (133.415, 55418.76176573929), (166.415, 43156.57866784828), (188.615, 27822.532710565378), (197.015, 28656.478468142588), (215.015, 33175.138989019964), (230.315, 29896.68314696604), (247.115, 41663.80742333608), (260.015, 34725.68558370795)]
DVBBS___Tsunami.wav 
 [(12.515, 35985.714880767715), (28.11

EDX___Feel_the_Rush.wav 
 [(6.815, 38370.875589566975), (34.115, 58978.381160283505), (48.515, 34622.11302796367), (80.615, 46579.71662496887), (87.215, 35062.86603007071), (97.715, 36378.17280711895), (106.115, 38989.56477593817), (114.815, 35872.603126659495), (142.715, 68389.81107681294), (154.115, 62326.800252047164), (168.515, 34613.592312296285), (179.315, 33775.83097750991)]
Dada_Life___Born_To_Rage.wav 
 [(21.515, 32350.602668738917), (34.415, 33179.27046661384), (51.515, 32267.935018081636), (65.915, 41968.438120509876), (83.315, 28748.286891188724), (97.115, 73478.80655265095), (109.115, 33086.57646872134), (116.615, 32557.236450452085), (131.315, 31980.07741854261), (145.715, 37505.11055209933), (174.515, 38903.12592285242), (188.615, 29372.706732158622), (206.615, 39696.15874270679)]
Tiesto___Boom.wav 
 [(14.615, 134354.50131013567), (27.215, 33322.27477939973), (38.615, 33880.35821070184), (51.815, 33382.94395161602), (60.815, 33621.8409184836), (79.415, 39951.029588373174

Zedd___The_Middle_(Alphalove_Remix).wav 
 [(12.515, 34819.83654203173), (29.915, 34086.0089026165), (43.715, 33940.128829741), (54.215, 32945.03744498521), (67.715, 44318.292365761714), (80.915, 32068.751684593546), (101.015, 33570.64773097453), (117.515, 33471.53749688138), (126.215, 33957.08073545419), (140.315, 33418.15599722359), (151.415, 48341.29546899899), (164.915, 32011.289102396902), (185.615, 37090.69904695122), (197.615, 33899.79603561286), (212.315, 47351.5907481083), (226.115, 32490.89363802908), (245.915, 34872.22966375738)]
Paul_Damixie___Get_Lost_(Extended_Version)_16.05.17_17.wav 
 [(10.415, 36190.62926209651), (22.415, 34416.7789649863), (35.615, 33193.19246195448), (43.415, 34278.64184457705), (50.315, 33086.709004137774), (62.915, 39750.57833376292), (69.815, 35124.23421286858), (78.215, 36036.883207729064), (88.715, 33790.25669825187), (107.915, 32665.92775677273), (126.815, 36775.296115291254), (153.215, 33363.21764321186), (171.515, 32627.340687440927), (191.015

Avicii___The_Nights.wav 
 [(7.715, 33914.017297736944), (28.715, 32126.238076358208), (43.115, 33422.0863919182), (61.715, 48624.25245542026), (80.015, 33145.10113557696), (93.515, 31142.073518020617), (106.115, 32396.608960133195), (119.315, 33605.08597215546), (137.315, 36492.37769220409), (152.015, 34402.97395797276)]
Marshmello___Anne_Marie___FRIENDS_(Borgeous_Remix).wav 
 [(6.515, 46717.98173763581), (20.015, 33936.094574274124), (26.915, 32660.30278968209), (37.415, 32433.381731183785), (56.315, 40410.186991739414), (75.215, 33311.031969217125), (90.215, 35988.65097614333), (100.415, 33691.90743360909), (115.415, 32715.743622876533), (131.615, 33016.45163089676), (148.115, 40278.65696429429), (167.015, 33289.87105526511), (188.015, 37266.837261203)]
Avicii___Wake_Me_Up.wav 
 [(6.815, 38234.451210204854), (20.015, 33068.273549054706), (27.515, 32803.369391684086), (38.015, 33650.624148442526), (50.315, 30992.024298921166), (57.815, 30848.499446248792), (68.315, 39031.893362446215)

Camila_Cabello___Havana_ft._Young_Thug_(Dim_Wilder_Remix).wav 
 [(13.415, 36051.63342891734), (45.215, 42583.000217344554), (60.515, 35346.14386643856), (76.115, 41384.97931292421), (84.815, 40121.356778276575), (95.915, 33398.92858817302), (113.015, 34400.010579274414), (145.115, 42432.97318411607), (160.415, 35818.3169796915), (175.715, 36343.874846635976)]
The_Weeknd___Can't_Feel_My_Face_(Martin_Garrix_Remix).wav 
 [(20.915, 37035.808296182564), (28.115, 32412.371380653312), (33.815, 32497.247157484722), (46.415, 32587.717212762782), (64.415, 31011.777938966556), (77.915, 38573.90771648382), (90.515, 36347.39189672043), (105.515, 36276.955827086174), (122.615, 43081.79409067267), (139.115, 33266.2319824545), (157.115, 32839.989080879204), (164.615, 30055.373707077244), (171.515, 31532.211614082913), (182.315, 31782.711117741048), (199.715, 38493.148260438815), (212.315, 36276.11674930095), (227.315, 36507.377434700844)]
Steve_Aoki,_Daddy_Yankee,_Play_N_Skillz,_Elvis_Crespo___Azukita

In [8]:
import csv
import pandas as pd

df = pd.read_csv("songs_fixed.csv")
list.sort(results, key = lambda x : x[0])

detection_count = 0
drop_count = 0
for i in range(len(results)):
    drops = str(df.iloc[[i]]["Drops"][i]).split(", ")
    peaks = results[i][1]
    temp_detection_count = 0
    songname = df.iloc[[i]]["Song Name"][i]
    drops_found = []
    print(drops, peaks)
    for d in drops:
        for p in peaks:
            dval = float(d)
            pval = float(p[0])
            if pval >= dval - 5 and pval <= dval + 5:
                temp_detection_count += 1
                drops_found.append(d)
                break
    if temp_detection_count < len(drops):
        print(songname)
        print(drops_found)
        print(results[i][0])
    drop_count += len(drops)
    detection_count += temp_detection_count

print(detection_count / drop_count)

ZeroDivisionError: division by zero

In [12]:
song = "Alan_Walker___The_Spectre.wav"

mfccs = load_song(SONG_DIR + song)
X, Y = continuous_prob_estimate(mfccs)


(20632, 16)


In [ ]:
make_peak_plot(X, Y, n=3)